In [14]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/pip-req-build-tcumavgi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/pip-req-build-tcumavgi
  Resolved https://github.com/huggingface/transformers.git to commit 05260a1fc1c8571a2b421ce72b680d5f1bc3e5a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10290584 sha256=59e4bcb3bf1048fc8971001a9c5ab9ad3b87aec7b96738d60aacbcfcbb7be585
  Stored in directory: /private/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/pip-ephem-wheel-cache-i58jnzk8/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: transformers
    Found exi

In [1]:
import pandas as pd 
from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, ModernBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

/Users/emrekuru/Developer/Fine-Tune/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def prepare_datasets(tokenizer, dataset_name="financial_phrasebank", subset_name="sentences_50agree", max_length=128, random_state=42):
    # Load the dataset
    dataset = load_dataset(dataset_name, subset_name, trust_remote_code=True)
    
    # Convert to Pandas DataFrame
    df = pd.DataFrame(dataset['train'])

    # Stratify split into train, validation, and test
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df['sentence'], df['label'], test_size=0.2, stratify=df['label'], random_state=random_state
    )
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_texts, train_labels, test_size=0.1, stratify=train_labels, random_state=random_state
    )

    # Create DataFrames for each split
    train_df = pd.DataFrame({'sentence': train_texts, 'label': train_labels})
    val_df = pd.DataFrame({'sentence': val_texts, 'label': val_labels})
    test_df = pd.DataFrame({'sentence': test_texts, 'label': test_labels})

    # Convert DataFrames to Hugging Face Dataset format
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Define tokenization function
    def tokenize_function(example):
        return tokenizer(
            example["sentence"], 
            padding="max_length", 
            truncation=True, 
            max_length=max_length
        )

    # Tokenize datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Remove raw text and prepare for Hugging Face Trainer
    train_dataset = train_dataset.remove_columns(["sentence"])
    val_dataset = val_dataset.remove_columns(["sentence"])
    test_dataset = test_dataset.remove_columns(["sentence"])

    train_dataset = train_dataset.rename_column("label", "labels")
    val_dataset = val_dataset.rename_column("label", "labels")
    test_dataset = test_dataset.rename_column("label", "labels")

    train_dataset.set_format("torch")
    val_dataset.set_format("torch")
    test_dataset.set_format("torch")

    return train_dataset, val_dataset, test_dataset

In [3]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}


def get_training_args(MODEL_NAME):
    
    training_args = TrainingArguments(
        output_dir=f"./results/{MODEL_NAME}",        # Directory to save checkpoints
        evaluation_strategy="epoch",                # Evaluate at the end of each epoch
        learning_rate=5e-5,                         # Typical learning rate for BERT
        per_device_train_batch_size=16,             # Adjust based on hardware
        per_device_eval_batch_size=16,    
        num_train_epochs=3,                         # Number of training epochs
        weight_decay=0.01,                          # Regularization
        logging_dir=f"./logs/{MODEL_NAME}",         # Directory for logs
        logging_steps=10,                           # Log every 10 steps
        save_total_limit=2,                         # Limit number of saved checkpoints
        save_strategy="epoch",                      # Save at the end of each epoch
        report_to=["tensorboard"],                  # Enable TensorBoard logging
        load_best_model_at_end=True,                # Automatically load the best model at the end
    )

    return training_args

## ModernBERT

In [4]:
model_name = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = ModernBertForSequenceClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=3  
)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = get_training_args("answerdotai/ModernBERT-base")
train_dataset, val_dataset, test_dataset = prepare_datasets(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/Users/emrekuru/Developer/Fine-Tune/.venv/lib/python3.12/site-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Map: 100%|██████████| 970/970 [00:00<00:00, 28950.11 examples/s]
/var/folders/mv/_dg3pqgn2zdf7f95_1dg07rw0000gn/T/ipykernel_14850/3436064775.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                       
  0%|          | 0/654 [00:26<?, ?it/s]         

{'loss': 0.9879, 'grad_norm': 5.624727725982666, 'learning_rate': 4.923547400611621e-05, 'epoch': 0.05}


                                       
  0%|          | 0/654 [00:37<?, ?it/s]         

{'loss': 0.766, 'grad_norm': 3.766676902770996, 'learning_rate': 4.847094801223242e-05, 'epoch': 0.09}


                                       
  0%|          | 0/654 [00:48<?, ?it/s]         

...






























































                                       
                                              

  0%|          | 0/654 [13:55<?, ?it/s]        



{'eval_loss': 0.7690773606300354, 'eval_accuracy': 0.8536082474226804, 'eval_runtime': 25.3949, 'eval_samples_per_second': 38.197, 'eval_steps_per_second': 2.402, 'epoch': 3.0}


                                       
100%|██████████| 654/654 [13:43<00:00,  1.26s/it]

{'train_runtime': 823.3679, 'train_samples_per_second': 12.709, 'train_steps_per_second': 0.794, 'train_loss': 0.25571310659158486, 'epoch': 3.0}


TrainOutput(global_step=654, training_loss=0.25571310659158486, metrics={'train_runtime': 823.3679, 'train_samples_per_second': 12.709, 'train_steps_per_second': 0.794, 'total_flos': 891428705132544.0, 'train_loss': 0.25571310659158486, 'epoch': 3.0})

In [7]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
pd.DataFrame(pred_labels, columns=["prediction"]).to_csv("predictions/ModernBERT.csv", index=False)

100%|██████████| 61/61 [00:22<00:00,  2.71it/s]


## BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3
)

train_dataset, val_dataset, test_dataset = prepare_datasets(tokenizer)

In [ ]:
training_args = get_training_args("bert-base-uncased")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
pd.DataFrame(pred_labels, columns=["prediction"]).to_csv("predictions/BERT.csv", index=False)

## LoRA

In [ ]:
training_args = get_training_args("lora")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3
)

train_dataset, val_dataset, test_dataset = prepare_datasets(tokenizer)

In [ ]:
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    target_modules=["query", "value"],  # Apply LoRA to attention layers
    lora_dropout=0.1,  # Dropout rate
    bias="none",  # Options: "none", "all", or "lora_only"
    task_type="SEQ_CLS"  # Task type: Sequence Classification
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
pd.DataFrame(pred_labels, columns=["prediction"]).to_csv("predictions/LoRA.csv", index=False)

## Distil-BERT

In [ ]:
training_args = get_training_args("distilbert")

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3
)

train_dataset, val_dataset, test_dataset = prepare_datasets(tokenizer)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
pd.DataFrame(pred_labels, columns=["prediction"]).to_csv("predictions/DistilBERT.csv", index=False)

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
pd.DataFrame(pred_labels, columns=["prediction"]).to_csv("predictions/T5.csv", index=False)